<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/businessdayconventions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

In [ ]:
import QuantLib as ql
import unittest
import inspect # Used for generating informative test names or messages

# Helper to represent the test cases cleanly
class SingleCase:
    def __init__(self, calendar_instance, convention, start_date, period, end_of_month, expected_result):
        self.calendar = calendar_instance
        self.convention = convention
        self.start_date = start_date
        self.period = period
        self.end_of_month = end_of_month
        self.expected_result = expected_result

    def __str__(self):
        # Provides a string representation for debugging or messages
        return (f"Calendar: {self.calendar.name()}, Conv: {self.convention}, Start: {self.start_date}, "
                f"Period: {self.period}, EOM: {self.end_of_month}, Expected: {self.expected_result}")


class BusinessDayConventionTests(unittest.TestCase):

    def testConventions(self):
        """Testing business day conventions."""
        print("Testing business day conventions...")

        test_cases = [
            # Following
            SingleCase(ql.SouthAfrica(), ql.Following, ql.Date(3, ql.February, 2015), ql.Period(1, ql.Months), False, ql.Date(3, ql.March, 2015)),
            SingleCase(ql.SouthAfrica(), ql.Following, ql.Date(3, ql.February, 2015), ql.Period(4, ql.Days), False, ql.Date(9, ql.February, 2015)),
            SingleCase(ql.SouthAfrica(), ql.Following, ql.Date(31, ql.January, 2015), ql.Period(1, ql.Months), True, ql.Date(27, ql.February, 2015)), # Feb 28 is Sat -> Mar 2, but EOM=True pulls back
            SingleCase(ql.SouthAfrica(), ql.Following, ql.Date(31, ql.January, 2015), ql.Period(1, ql.Months), False, ql.Date(2, ql.March, 2015)), # Feb 28 is Sat -> Mar 2

            # ModifiedFollowing
            SingleCase(ql.SouthAfrica(), ql.ModifiedFollowing, ql.Date(3, ql.February, 2015), ql.Period(1, ql.Months), False, ql.Date(3, ql.March, 2015)),
            SingleCase(ql.SouthAfrica(), ql.ModifiedFollowing, ql.Date(3, ql.February, 2015), ql.Period(4, ql.Days), False, ql.Date(9, ql.February, 2015)),
            SingleCase(ql.SouthAfrica(), ql.ModifiedFollowing, ql.Date(31, ql.January, 2015), ql.Period(1, ql.Months), True, ql.Date(27, ql.February, 2015)), # Feb 28 is Sat -> Mar 2, ModFol -> Feb 27
            SingleCase(ql.SouthAfrica(), ql.ModifiedFollowing, ql.Date(31, ql.January, 2015), ql.Period(1, ql.Months), False, ql.Date(27, ql.February, 2015)),# Feb 28 is Sat -> Mar 2, ModFol -> Feb 27
            SingleCase(ql.SouthAfrica(), ql.ModifiedFollowing, ql.Date(25, ql.March, 2015), ql.Period(1, ql.Months), False, ql.Date(28, ql.April, 2015)), # Apr 25 -> Apr 27 (Mon Holiday) -> Apr 28. Still Apr, so ok.
            SingleCase(ql.SouthAfrica(), ql.ModifiedFollowing, ql.Date(7, ql.February, 2015), ql.Period(1, ql.Months), False, ql.Date(9, ql.March, 2015)), # Mar 7 is Sat -> Mar 9

            # Preceding
            SingleCase(ql.SouthAfrica(), ql.Preceding, ql.Date(3, ql.March, 2015), ql.Period(-1, ql.Months), False, ql.Date(3, ql.February, 2015)),
            SingleCase(ql.SouthAfrica(), ql.Preceding, ql.Date(3, ql.February, 2015), ql.Period(-2, ql.Days), False, ql.Date(30, ql.January, 2015)), # Feb 1 is Sun, Jan 31 is Sat -> Jan 30
            SingleCase(ql.SouthAfrica(), ql.Preceding, ql.Date(1, ql.March, 2015), ql.Period(-1, ql.Months), True, ql.Date(30, ql.January, 2015)), # Mar 1 Sun -> Feb 28 Sat -> Jan 31 Sat -> Jan 30 (EOM=True)
            SingleCase(ql.SouthAfrica(), ql.Preceding, ql.Date(1, ql.March, 2015), ql.Period(-1, ql.Months), False, ql.Date(30, ql.January, 2015)), # Feb 1 Sun -> Jan 31 Sat -> Jan 30

            # ModifiedPreceding
            SingleCase(ql.SouthAfrica(), ql.ModifiedPreceding, ql.Date(3, ql.March, 2015), ql.Period(-1, ql.Months), False, ql.Date(3, ql.February, 2015)),
            SingleCase(ql.SouthAfrica(), ql.ModifiedPreceding, ql.Date(3, ql.February, 2015), ql.Period(-2, ql.Days), False, ql.Date(30, ql.January, 2015)),
            SingleCase(ql.SouthAfrica(), ql.ModifiedPreceding, ql.Date(1, ql.March, 2015), ql.Period(-1, ql.Months), True, ql.Date(2, ql.February, 2015)), # Mar 1 Sun -> Feb 28 Sat -> Jan 31 Sat -> Jan 30. ModPrec -> Feb 2
            SingleCase(ql.SouthAfrica(), ql.ModifiedPreceding, ql.Date(1, ql.March, 2015), ql.Period(-1, ql.Months), False, ql.Date(2, ql.February, 2015)), # Feb 1 Sun -> Jan 31 Sat -> Jan 30. ModPrec -> Feb 2

            # Unadjusted
            SingleCase(ql.SouthAfrica(), ql.Unadjusted, ql.Date(3, ql.February, 2015), ql.Period(1, ql.Months), False, ql.Date(3, ql.March, 2015)),
            SingleCase(ql.SouthAfrica(), ql.Unadjusted, ql.Date(3, ql.February, 2015), ql.Period(4, ql.Days), False, ql.Date(7, ql.February, 2015)), # Feb 7 is Sat, Unadjusted stays Sat
            SingleCase(ql.SouthAfrica(), ql.Unadjusted, ql.Date(31, ql.January, 2015), ql.Period(1, ql.Months), True, ql.Date(28, ql.February, 2015)), # EOM=True respected
            SingleCase(ql.SouthAfrica(), ql.Unadjusted, ql.Date(31, ql.January, 2015), ql.Period(1, ql.Months), False, ql.Date(28, ql.February, 2015)), # Feb 28 is Sat

            # HalfMonthModifiedFollowing
            SingleCase(ql.SouthAfrica(), ql.HalfMonthModifiedFollowing, ql.Date(3, ql.February, 2015), ql.Period(1, ql.Months), False, ql.Date(3, ql.March, 2015)),
            SingleCase(ql.SouthAfrica(), ql.HalfMonthModifiedFollowing, ql.Date(3, ql.February, 2015), ql.Period(4, ql.Days), False, ql.Date(9, ql.February, 2015)),
            SingleCase(ql.SouthAfrica(), ql.HalfMonthModifiedFollowing, ql.Date(31, ql.January, 2015), ql.Period(1, ql.Months), True, ql.Date(27, ql.February, 2015)), # Feb 28 Sat -> Mar 2. >15th, ModFol -> Feb 27
            SingleCase(ql.SouthAfrica(), ql.HalfMonthModifiedFollowing, ql.Date(31, ql.January, 2015), ql.Period(1, ql.Months), False, ql.Date(27, ql.February, 2015)),# Feb 28 Sat -> Mar 2. >15th, ModFol -> Feb 27
            SingleCase(ql.SouthAfrica(), ql.HalfMonthModifiedFollowing, ql.Date(3, ql.January, 2015), ql.Period(1, ql.Weeks), False, ql.Date(12, ql.January, 2015)), # Jan 10 Sat -> Jan 12. <=15th, Following
            SingleCase(ql.SouthAfrica(), ql.HalfMonthModifiedFollowing, ql.Date(21, ql.March, 2015), ql.Period(1, ql.Weeks), False, ql.Date(30, ql.March, 2015)), # Mar 28 Sat -> Mar 30. >15th, Following
            SingleCase(ql.SouthAfrica(), ql.HalfMonthModifiedFollowing, ql.Date(7, ql.February, 2015), ql.Period(1, ql.Months), False, ql.Date(9, ql.March, 2015)), # Mar 7 Sat -> Mar 9. <=15th, Following

            # Nearest (equivalent to WeekendsOnly adjustment in some contexts, but Nearest exists as enum)
            SingleCase(ql.SouthAfrica(), ql.Nearest, ql.Date(3, ql.February, 2015), ql.Period(1, ql.Months), False, ql.Date(3, ql.March, 2015)), # In range, no adjustment needed
            SingleCase(ql.SouthAfrica(), ql.Nearest, ql.Date(3, ql.February, 2015), ql.Period(4, ql.Days), False, ql.Date(9, ql.February, 2015)), # Feb 7 Sat -> Nearest Friday (Feb 6) or Monday (Feb 9)? QL's Nearest usually means nearest *working* day. Feb 7 is Sat. Nearest WD is Mon Feb 9.
                                                                                                                                                # Wait, C++ expected is Feb 9. Let's re-read doc: "Adjust to nearest business day. If the non-business day falls on a Monday or Tuesday, adjust to the previous business day. If it falls on a Wednesday, Thursday or Friday, adjust to the next business day."
                                                                                                                                                # This definition seems specific to some markets. QL `Nearest` enum description is usually just "choose the nearest business day".
                                                                                                                                                # Let's test QL behavior: ql.SouthAfrica().advance(ql.Date(3,2,2015), ql.Period("4d"), ql.Nearest) -> Feb 9, 2015. So QL's `Nearest` behaves as 'nearest WD'.
            SingleCase(ql.SouthAfrica(), ql.Nearest, ql.Date(16, ql.April, 2015), ql.Period(1, ql.Months), False, ql.Date(15, ql.May, 2015)), # May 16 is Sat. Nearest WD is Fri May 15.
            SingleCase(ql.SouthAfrica(), ql.Nearest, ql.Date(17, ql.April, 2015), ql.Period(1, ql.Months), False, ql.Date(18, ql.May, 2015)), # May 17 is Sun. Nearest WD is Mon May 18.
            SingleCase(ql.SouthAfrica(), ql.Nearest, ql.Date(4, ql.March, 2015), ql.Period(1, ql.Months), False, ql.Date(2, ql.April, 2015)), # Apr 4 is Sat. Nearest WD is Fri Apr 3? No, wait, Apr 3 is Good Friday. So nearest is Thu Apr 2.
            SingleCase(ql.SouthAfrica(), ql.Nearest, ql.Date(2, ql.April, 2015), ql.Period(1, ql.Months), False, ql.Date(4, ql.May, 2015)),  # May 2 is Sat. Nearest WD is Mon May 4.
        ]

        for i, case in enumerate(test_cases):
            # Get a fresh calendar instance for safety, though SouthAfrica() should be stateless
            calendar = case.calendar

            # Use the advance method from the calendar instance
            result = calendar.advance(
                case.start_date,
                case.period,
                case.convention,
                case.end_of_month)

            # Check if the result matches the expected result
            self.assertEqual(result, case.expected_result,
                             msg=(f"\ncase {i}:\n"
                                  f"{case}\n"
                                  f"convention: {case.convention}\n"
                                  f"actual result: {result}"))

if __name__ == '__main__':
    print("Presolve testQuantLib.py ...")
    unittest.main(argv=['first-arg-is-ignored'], exit=False)